# Right Left up down, motion vectors v4

## Setup

In [1]:
import os
from pathlib import Path

import tensorflow as tf
import keras
import pandas as pd

from frame_generator_mv_v4 import FrameGenerator
from model_mv_v4 import create_R2Plus1D_mv_model

2024-01-08 11:09:21.187981: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-08 11:09:22.796093: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-08 11:09:22.796158: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-08 11:09:24.018294: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 11:09:34.289826: W tensorflow/stream_executor/platform/de

In [3]:
DATA_PATH = "data_right_left_up_down_1200"
NOTEBOOK_NAME = "4.2_2d_plus_1_rlud_1200_mv"
RESULTS_PATH = DATA_PATH + "/" + NOTEBOOK_NAME

index_df = pd.read_csv(f'{DATA_PATH}/indx_df.csv')

# PARAMS
n_frames = 74
batch_size = 8
HEIGHT=420
WIDTH = 10
MOTION_CHANNELS=1 # or 10 

assert(os.path.isdir(DATA_PATH ))

if not os.path.isdir(RESULTS_PATH):
    os.mkdir(RESULTS_PATH)

index_df = pd.read_csv(f'{DATA_PATH}/indx_df.csv')

print(f"classes being compared {index_df['category'].unique()}")

classes being compared ['Pushing something from left to right'
 'Pushing something from right to left' 'Moving something up'
 'Moving something down']


## Preprocess video data

Load something something data tf.data.Dataset

In [4]:
subset_paths = {
    "test": Path(f'{DATA_PATH}/test'),
    "train": Path(f'{DATA_PATH}/train'),
    "val": Path(f'{DATA_PATH}/validation'),
}

output_signature = (
    tf.TensorSpec(shape = (n_frames, HEIGHT, WIDTH), dtype = tf.float32),
    tf.TensorSpec(shape = (), dtype = tf.int16)
)
train_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        height=HEIGHT,
        index_df=index_df,
        training=True
    ),
    output_signature = output_signature
)
# Batch the data
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        height=HEIGHT,
        index_df=index_df,
        training=True
    ),
    output_signature = output_signature
)
# Batch the data
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        height=HEIGHT,
        index_df=index_df,
        training=True
    ),
    output_signature = output_signature
)

# Batch the data
test_ds = test_ds.batch(batch_size)


<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


## Model Creation


In [5]:
input_shape = (n_frames, HEIGHT, WIDTH, MOTION_CHANNELS)
model = create_R2Plus1D_mv_model(input_shape, num_classes=4)

## Validate Model and Data

In [6]:
# testing tensor is setup correct
iter(train_ds)

## Build Model

In [7]:
frames, label = next(iter(train_ds))

In [8]:
model.build(frames)

## Visualise Model

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 74, 420, 10  0           []                               
                                , 1)]                                                             
                                                                                                  
 time_distributed (TimeDistribu  (None, 74, 420, 10,  640        ['input_1[0][0]']                
 ted)                            64)                                                              
                                                                                                  
 time_distributed_1 (TimeDistri  (None, 74, 420, 10,  256        ['time_distributed[0][0]']       
 buted)                          64)                                                          

## Load the Model

Using BinaryCrossentropy as it is more effective for binary data

from_logits is false because final layer includes a sigmoid activation,

In [16]:
previous_runs = 3

In [17]:
previous_runs += 1

model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
              metrics=[
                    'accuracy',
                    # tf.keras.metrics.Precision(),
                    # tf.keras.metrics.Recall()
                    # tf.keras.metrics.Precision(class_id=0, name='precision_neg'),
                    # tf.keras.metrics.Precision(class_id=1, name='precision_pos'),
                    # tf.keras.metrics.Recall(class_id=0, name='recall_neg'),
                    # tf.keras.metrics.Recall(class_id=1, name='recall_pos')
                ]
            )

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    RESULTS_PATH + '/model-runs-' + str(previous_runs) + '-cp-{epoch:02d}-{val_loss:.2f}.ckpt',
    save_best_only=True,  # Save only the best model based on a monitored metric (e.g., val_loss), will only replace saved value if it is better
    monitor='val_loss',
    mode='min',  # 'min' for loss, 'max' for accuracy
    save_weights_only=True
)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True
)

In [18]:
latest = tf.train.latest_checkpoint(RESULTS_PATH)

if latest is not None:
    print(f"loading model from weights: {latest}")
    model.load_weights(latest)

loading model from weights: data_right_left_up_down_1200/2_2d_plus_1_rlud_1200_mv_v4/model-runs-3-cp-07-0.19.ckpt


## Train the Model

In [19]:
previously_run_epochs = 0 + 16 + 15 + 16

history = model.fit(
        x=train_ds,
        epochs = 50 - previously_run_epochs,
        validation_data=val_ds,
        callbacks=[checkpoint_callback, early_stopping],
    )

Epoch 1/3
480/480 [==============================] - 1275s 3s/step - loss: 0.2899 - accuracy: 0.8987 - val_loss: 0.2267 - val_accuracy: 0.9242
Epoch 2/3
480/480 [==============================] - 1281s 3s/step - loss: 0.2989 - accuracy: 0.8992 - val_loss: 0.1901 - val_accuracy: 0.9427
Epoch 3/3
480/480 [==============================] - 1280s 3s/step - loss: 0.2923 - accuracy: 0.8990 - val_loss: 0.5920 - val_accuracy: 0.7875


## Analyse results

## Analyse results

In [14]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

true_labels = []
predictions = []

# Iterate over the batched test dataset
for batch in test_ds:
    x, y = batch  # x is the batch of features, y is the batch of labels
    true_labels.extend(y.numpy())  # Store true labels
    preds = model.predict(x)  # Generate predictions for the batch
    preds = softmax(preds, axis=1)  # Apply softmax to convert logits to probabilities
    preds = np.argmax(preds, axis=1)  # Get the class with the highest probability
    predictions.extend(preds)

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predictions = np.array(predictions)

1/1 [==============================] - 0s 29ms/step


In [15]:
fg = FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        height=HEIGHT,
        index_df=index_df,
        training=True
)
class_id_value = {
    fg.class_ids_for_name[x]: x for x in fg.class_ids_for_name.keys()
 }

# Convert lists to numpy arrays if they aren't already
true_labels = np.array(true_labels)
predictions = np.array(predictions)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)

# Calculate precision, recall, and F1-score for each class
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average=None)

# Print accuracy and F1-scores for each class
print(f"Overall Accuracy: {accuracy}")
for i, (prec, rec, f1) in enumerate(zip(precision, recall, f1_score)):
    print(f"Class {class_id_value[i]}: Precision: {prec}, Recall: {rec}, F1 Score: {f1}")

Overall Accuracy: 0.94453125
Class Pushing something from left to right: Precision: 0.9681190223166843, Recall: 0.9489583333333333, F1 Score: 0.9584429247764334
Class Pushing something from right to left: Precision: 0.96723044397463, Recall: 0.953125, F1 Score: 0.9601259181532004
Class Moving something up: Precision: 0.8838095238095238, Recall: 0.9666666666666667, F1 Score: 0.9233830845771145
Class Moving something down: Precision: 0.9667774086378738, Recall: 0.909375, F1 Score: 0.9371980676328503
